In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 드라이버 설정
driver = webdriver.Chrome()
url = "https://www.dongwonfnb.com/services/Product/Product_List"
driver.get(url)

# 첫 로딩 대기
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "dl.prodMenu"))
)

page_num = 1

while True:
    
    print(f"📄 현재 페이지: {page_num}")
    time.sleep(1)

    try:
        pagination = driver.find_element(By.CLASS_NAME, "boardPage")
        li_list = pagination.find_elements(By.TAG_NAME, "li")

        # 현재 페이지 인덱스 찾기
        current_idx = -1
        for i, li in enumerate(li_list):
            if "on" in li.get_attribute("class"):
                current_idx = i
                break

        next_found = False

        # 다음 숫자 페이지가 있으면 클릭
        if 0 <= current_idx < len(li_list) - 1:
            next_li = li_list[current_idx + 1]
            if "btn_next" not in next_li.get_attribute("class"):
                next_li.find_element(By.TAG_NAME, "a").click()
                page_num += 1
                next_found = True
                continue

        # 다음 페이지 세트 (btn_next) 클릭
        if not next_found:
            try:
                prev_page = page_num

                next_btn = driver.find_element(By.CSS_SELECTOR, "ul.boardNav > li > a.btn_next")
                driver.execute_script("arguments[0].click();", next_btn)
                time.sleep(2)  # 페이지 전환 대기

                # 새로운 페이지 번호 체크
                pagination = driver.find_element(By.CLASS_NAME, "boardPage")
                li_list = pagination.find_elements(By.TAG_NAME, "li")

                for i, li in enumerate(li_list):
                    if "on" in li.get_attribute("class"):
                        page_num = int(li.text.strip())
                        break

                if page_num == prev_page:
                    print("✅ 마지막 페이지 도달 (btn_next 클릭해도 변화 없음).")
                    break

                print("➡️ 다음 버튼 클릭 (페이지 세트 이동)")
                continue
            except Exception as e:
                print(f"❗ 다음 버튼 클릭 중 에러: {e}")
                break

    except Exception as e:
        print(f"❗ 페이지 이동 중 에러 발생: {e}")
        break

driver.quit()
print("✅ 페이지 이동 테스트 종료.")


📄 현재 페이지: 1
📄 현재 페이지: 2
📄 현재 페이지: 3
📄 현재 페이지: 4
📄 현재 페이지: 5
📄 현재 페이지: 6
📄 현재 페이지: 7
📄 현재 페이지: 8
📄 현재 페이지: 9
📄 현재 페이지: 10
➡️ 다음 버튼 클릭 (페이지 세트 이동)
📄 현재 페이지: 11
📄 현재 페이지: 12
📄 현재 페이지: 13
📄 현재 페이지: 14
✅ 마지막 페이지 도달 (btn_next 클릭해도 변화 없음).
✅ 페이지 이동 테스트 종료.


In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# 드라이버 설정
driver = webdriver.Chrome()
url = "https://www.dongwonfnb.com/services/Product/Product_List"
driver.get(url)

# 결과 저장 리스트
results = []

# 첫 로딩 대기
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "dl.prodMenu"))
)

page_num = 1

while True:
    print(f"📄 현재 페이지: {page_num}")
    time.sleep(2)

    # 제품 블록들 수집
    try:
        blocks = driver.find_elements(By.CSS_SELECTOR, "dl.prodMenu")
        for block in blocks:
            try:
                product_name = block.find_element(By.TAG_NAME, "dt").get_attribute("textContent").strip()

                tag_wrap = block.find_elements(By.CSS_SELECTOR, "div.prodTag_wrap > span")
                tags = [t.get_attribute("textContent").strip() for t in tag_wrap]

                results.append({
                    "제품명": product_name,
                    "태그": ", ".join(tags) if tags else ""
                })
            except Exception as e:
                print(f"❗ 제품 정보 파싱 에러: {e}")
    except Exception as e:
        print(f"❗ 블록 수집 에러: {e}")
        break

    # 페이지 이동
    try:
        pagination = driver.find_element(By.CLASS_NAME, "boardPage")
        li_list = pagination.find_elements(By.TAG_NAME, "li")

        # 현재 페이지 인덱스 확인
        current_idx = -1
        for i, li in enumerate(li_list):
            if "on" in li.get_attribute("class"):
                current_idx = i
                break

        next_found = False

        # 다음 숫자 페이지 클릭
        if 0 <= current_idx < len(li_list) - 1:
            next_li = li_list[current_idx + 1]
            if "btn_next" not in next_li.get_attribute("class"):
                next_li.find_element(By.TAG_NAME, "a").click()
                page_num += 1
                continue

        # 다음 세트 페이지 (btn_next) 클릭
        prev_page = page_num
        next_btn = driver.find_element(By.CSS_SELECTOR, "ul.boardNav > li > a.btn_next")
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(2)

        # 새 페이지 번호 업데이트
        pagination = driver.find_element(By.CLASS_NAME, "boardPage")
        li_list = pagination.find_elements(By.TAG_NAME, "li")
        for i, li in enumerate(li_list):
            if "on" in li.get_attribute("class"):
                page_num = int(li.text.strip())
                break

        if page_num == prev_page:
            print("✅ 마지막 페이지 도달 (btn_next 클릭해도 이동 없음)")
            break

    except Exception as e:
        print(f"❗ 페이지 이동 중 에러 발생: {e}")
        break

driver.quit()

# CSV 저장
df = pd.DataFrame(results)
df.to_csv("dongwonfnb_product_list_with_tags.csv", index=False, encoding="utf-8-sig")
print("✅ 전체 제품 정보 저장 완료. 총 수집 개수:", len(df))


📄 현재 페이지: 1
📄 현재 페이지: 2
📄 현재 페이지: 3
📄 현재 페이지: 4
📄 현재 페이지: 5
📄 현재 페이지: 6
📄 현재 페이지: 7
📄 현재 페이지: 8
📄 현재 페이지: 9
📄 현재 페이지: 10
📄 현재 페이지: 11
📄 현재 페이지: 12
📄 현재 페이지: 13
📄 현재 페이지: 14
✅ 마지막 페이지 도달 (btn_next 클릭해도 이동 없음)
✅ 전체 제품 정보 저장 완료. 총 수집 개수: 335


In [ ]:
# dongwonfnb_scraper_final.py

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# ===== 드라이버 설정 =====
driver = webdriver.Chrome()
driver.get("https://www.dongwonfnb.com/services/Product/Product_List")
wait = WebDriverWait(driver, 10)
time.sleep(3)

results = []

# ===== 1차 카테고리 수집 =====
first_select = Select(wait.until(EC.presence_of_element_located((By.ID, "selCategoryD1"))))
first_options = [opt.text for opt in first_select.options if opt.text != "1차분류"]

for first in first_options:
    # fresh select 객체로 1차 카테고리 선택
    first_select = Select(driver.find_element(By.ID, "selCategoryD1"))
    first_select.select_by_visible_text(first)
    time.sleep(2)

    # ===== 2차 카테고리 수집 =====
    second_select = Select(driver.find_element(By.ID, "selCategoryD2"))
    second_options = [opt.text for opt in second_select.options if opt.text != "2차분류"]

    for second in second_options:
        # fresh select 객체로 2차 카테고리 선택
        second_select = Select(driver.find_element(By.ID, "selCategoryD2"))
        second_select.select_by_visible_text(second)
        time.sleep(1)

        # ✅ "조회" 버튼 클릭
        search_btn = driver.find_element(By.CLASS_NAME, "ico_right")
        search_btn.click()
        time.sleep(2)

        # ===== 페이지 루프 시작 =====
        page_num = 1
        while True:
            print(f"📄 {first} > {second} > 페이지 {page_num}")
            time.sleep(1)

            items = driver.find_elements(By.CSS_SELECTOR, "dl.prodMenu > dt")
            for item in items:
                results.append({
                    "1차카테고리": first,
                    "2차카테고리": second,
                    "메뉴명": item.get_attribute("textContent").strip()
                })

            # 다음 페이지 버튼 탐색
            try:
                pagination = driver.find_element(By.CLASS_NAME, "boardPage")
                li_list = pagination.find_elements(By.TAG_NAME, "li")

                current_idx = -1
                for i, li in enumerate(li_list):
                    if "on" in li.get_attribute("class"):
                        current_idx = i
                        break

                next_found = False
                if 0 <= current_idx < len(li_list) - 1:
                    next_li = li_list[current_idx + 1]
                    if "btn_next" not in next_li.get_attribute("class"):
                        next_li.find_element(By.TAG_NAME, "a").click()
                        page_num += 1
                        next_found = True
                        continue

                # 다음 세트 버튼 처리
                if not next_found:
                    try:
                        prev_page = page_num
                        next_btn = driver.find_element(By.CSS_SELECTOR, "ul.boardNav > li > a.btn_next")
                        driver.execute_script("arguments[0].click();", next_btn)
                        time.sleep(2)

                        # 페이지 증가했는지 확인
                        pagination = driver.find_element(By.CLASS_NAME, "boardPage")
                        li_list = pagination.find_elements(By.TAG_NAME, "li")
                        for li in li_list:
                            if "on" in li.get_attribute("class"):
                                page_num = int(li.text.strip())
                                break

                        if page_num == prev_page:
                            print("✅ 마지막 페이지 도달.")
                            break

                        print("➡️ 다음 버튼 클릭 (세트 이동)")
                        continue
                    except:
                        print("✅ 다음 세트 없음 (btn_next 클릭 실패).")
                        break

            except Exception as e:
                print(f"❗ 페이지 이동 중 에러: {e}")
                break

driver.quit()

# ===== 결과 저장 =====
df = pd.DataFrame(results)
df.to_csv("dongwonfnb_all_products.csv", index=False, encoding="utf-8-sig")
print("✅ 제품 목록 저장 완료. 총 항목 수:", len(df))


📄 참치/수산 > 참치캔 > 페이지 1
📄 참치/수산 > 참치캔 > 페이지 2
✅ 마지막 페이지 도달.
📄 참치/수산 > 수산캔 > 페이지 1
✅ 마지막 페이지 도달.
📄 축산 > 축산캔 > 페이지 1
✅ 마지막 페이지 도달.
📄 과일농산 > 과일농산캔 > 페이지 1
✅ 마지막 페이지 도달.
📄 즉석밥/죽/국탕찌개 > 즉석밥 > 페이지 1
✅ 마지막 페이지 도달.
📄 즉석밥/죽/국탕찌개 > 죽 > 페이지 1
✅ 마지막 페이지 도달.
📄 즉석밥/죽/국탕찌개 > 국탕찌개 > 페이지 1
✅ 마지막 페이지 도달.
📄 해조류 > 김 > 페이지 1
✅ 마지막 페이지 도달.
📄 해조류 > 미역 > 페이지 1
✅ 마지막 페이지 도달.
📄 김치 > 김치 > 페이지 1
✅ 마지막 페이지 도달.
📄 음료/샘물 > 녹차/티백 > 페이지 1
✅ 마지막 페이지 도달.
📄 음료/샘물 > 커피 > 페이지 1
✅ 마지막 페이지 도달.
📄 음료/샘물 > 샘물 > 페이지 1
✅ 마지막 페이지 도달.
📄 음료/샘물 > 기타음료 > 페이지 1
✅ 마지막 페이지 도달.
📄 냉장햄 > 냉장햄 > 페이지 1
✅ 마지막 페이지 도달.
📄 맛살/어묵 > 맛살 > 페이지 1
✅ 마지막 페이지 도달.
📄 맛살/어묵 > 어묵 > 페이지 1
✅ 마지막 페이지 도달.
📄 냉동 > 안주류 > 페이지 1
✅ 마지막 페이지 도달.
📄 냉동 > 만두 > 페이지 1
✅ 마지막 페이지 도달.
📄 냉동 > 돈까스/간식류 > 페이지 1
✅ 마지막 페이지 도달.
📄 냉동 > 완자/동그랑땡 > 페이지 1
✅ 마지막 페이지 도달.
📄 과일쨈 > 과일쨈 > 페이지 1
✅ 마지막 페이지 도달.
📄 조미/오일 > 오일 > 페이지 1
✅ 마지막 페이지 도달.
📄 조미/오일 > 조미 > 페이지 1
✅ 마지막 페이지 도달.
📄 면류 > 생/건면류 > 페이지 1
✅ 마지막 페이지 도달.
📄 분식류 > 떡볶이 > 페이지 1
✅ 마지막 페이지 도달.
📄 분식류 > 면 > 페이지 1
✅ 마지막 페이지 도달.
📄 디저트 > 디저트 > 페이지 1
✅ 마지

In [4]:
driver.find_elements(By.CSS_SELECTOR, ".boardPage li")


[<selenium.webdriver.remote.webelement.WebElement (session="571e43490ff4b1f6396e490f3c197525", element="f.463D0A388842A1B6203597A0C449AAA6.d.DC624B4057E0D37C59B64AE3FBD3C63F.e.1106")>,
 <selenium.webdriver.remote.webelement.WebElement (session="571e43490ff4b1f6396e490f3c197525", element="f.463D0A388842A1B6203597A0C449AAA6.d.DC624B4057E0D37C59B64AE3FBD3C63F.e.1108")>,
 <selenium.webdriver.remote.webelement.WebElement (session="571e43490ff4b1f6396e490f3c197525", element="f.463D0A388842A1B6203597A0C449AAA6.d.DC624B4057E0D37C59B64AE3FBD3C63F.e.1110")>,
 <selenium.webdriver.remote.webelement.WebElement (session="571e43490ff4b1f6396e490f3c197525", element="f.463D0A388842A1B6203597A0C449AAA6.d.DC624B4057E0D37C59B64AE3FBD3C63F.e.1112")>,
 <selenium.webdriver.remote.webelement.WebElement (session="571e43490ff4b1f6396e490f3c197525", element="f.463D0A388842A1B6203597A0C449AAA6.d.DC624B4057E0D37C59B64AE3FBD3C63F.e.1114")>,
 <selenium.webdriver.remote.webelement.WebElement (session="571e43490ff4b1f

In [5]:
driver.find_element(By.LINK_TEXT, "2").click()  # 또는 위에서 li → a 접근


In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# ChromeDriver 경로 설정
chrome_path = "C:/Users/baby3/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"  # 사용자 맞게 수정

# Chrome 옵션 설정 (선택)
options = Options()
options.add_argument("--start-maximized")
# options.add_argument("--headless")  # 창 없이 실행하려면 주석 해제

# 웹드라이버 실행
driver = webdriver.Chrome()
driver.get("https://www.dongwonmall.com/category/main.do?cate_id=01110065")

# "리뷰 많은순" 클릭
try:
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.sort")))
    sort_buttons = driver.find_elements(By.CSS_SELECTOR, "span.sort")
    for btn in sort_buttons:
        if btn.text.strip() == "리뷰 많은순":
            btn.click()
            break
except Exception as e:
    print("리뷰 많은순 클릭 오류:", e)

# 상품 리스트 로딩 대기
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "ul#bandBestList > li.product_item"))
)
time.sleep(2)  # 동적 로딩 안정화용 잠깐 대기

# 최신 DOM에서 다시 상품 목록 수집
items = driver.find_elements(By.CSS_SELECTOR, "ul#bandBestList > li.product_item")

# 데이터 수집
product_list = []
for item in items:
    try:
        name = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > a > p.name").text.strip()
    except:
        name = "N/A"
    try:
        price = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > div.price_info > span.new_price > strong").text.strip()
    except:
        price = "N/A"
    try:
        grade = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > span.star_rating > span.grade").text.strip()
    except:
        grade = "N/A"
    try:
        review = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > span.star_rating > span.review").text.strip()
    except:
        review = "N/A"

    product_list.append({
        "상품명": name,
        "가격": price,
        "평점": grade,
        "리뷰 수": review
    })

# 브라우저 종료
driver.quit()

# 데이터프레임 출력
df = pd.DataFrame(product_list)
print(df)

# # 엑셀 저장 (선택)
df.to_excel("dongwonmall_review_sorted_products.xlsx", index=False)


                                   상품명       가격   평점      리뷰 수
0           동원참치 살코기 라이트스탠다드 85g x 20캔  31,900원    5  (2,299건)
1   동원 라이트스탠다드 참치 150gx10캔+리챔 200gx10캔  49,200원  4.9  (1,464건)
2           동원참치 살코기 라이트스탠다드 150g x 5캔  12,300원  4.9    (845건)
3          동원참치 살코기 라이트스탠다드 150g x 20캔  50,000원  4.9    (897건)
4           동원참치 살코기 라이트스탠다드 85g x 10캔  16,600원  4.9    (905건)
..                                 ...      ...  ...       ...
91                    동원 고등어 400g x 6캔  13,800원  4.8    (101건)
92                       동원참치 고추참치 85g   2,200원    5    (105건)
93      [4%적립금] 동원 고등어 400g x 24개(1박스)  45,900원    5    (121건)
94     동원 양반 소고기 미역국 460g (1~2인분) x 3개  12,000원  4.9    (101건)
95                동원 양반 단호박죽 420g x 4개  10,900원  4.8    (102건)

[96 rows x 4 columns]


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# 크롬 드라이버 경로 설정
chrome_path = "C:/Users/baby3/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"

# 크롬 옵션
options = Options()
options.add_argument("--start-maximized")

# 드라이버 실행
driver = webdriver.Chrome()
driver.get("https://www.dongwonmall.com/category/main.do?cate_id=01110065")

# "리뷰 많은순" 클릭
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.sort")))
sort_buttons = driver.find_elements(By.CSS_SELECTOR, "span.sort")
for btn in sort_buttons:
    if btn.text.strip() == "리뷰 많은순":
        btn.click()
        break

time.sleep(2)

product_list = []
current_page = 1
max_pages = 2

while True:
    print(f"[페이지 {current_page}] 수집 중...")

    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul#bandBestList > li.product_item"))
    )
    time.sleep(1)

    items = driver.find_elements(By.CSS_SELECTOR, "ul#bandBestList > li.product_item")
    for item in items:
        try:
            name = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > a > p.name").text.strip()
        except:
            name = "N/A"
        try:
            price = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > div.price_info > span.new_price > strong").text.strip()
        except:
            price = "N/A"
        try:
            grade = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > span.star_rating > span.grade").text.strip()
        except:
            grade = "N/A"
        try:
            review = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > span.star_rating > span.review").text.strip()
        except:
            review = "N/A"

        product_list.append({
            "페이지": current_page,
            "상품명": name,
            "가격": price,
            "평점": grade,
            "리뷰 수": review
        })

    if current_page >= max_pages:
        print("✅ 최대 페이지(10) 도달, 종료")
        break

    try:
        # 다음 페이지 번호 존재하는지 확인 (paging 내부의 ul > li > a 구조 사용)
        page_selector = f'div.paging.paging_t20r > ul > li > a'
        all_pages = driver.find_elements(By.CSS_SELECTOR, page_selector)

        next_page_number = str(current_page + 1)
        next_page_elem = None
        for p in all_pages:
            if p.text.strip() == next_page_number:
                next_page_elem = p
                break

        if next_page_elem:
            driver.execute_script("arguments[0].click();", next_page_elem)
            current_page += 1
            time.sleep(2)
        else:
            print("✅ 다음 페이지 없음, 종료")
            break
    except Exception as e:
        print("❌ 예외 발생:", e)
        break

# 저장 및 종료
driver.quit()

df = pd.DataFrame(product_list)
df.to_excel("dongwonmall_review_auto_paging.xlsx", index=False)
print("✅ 엑셀 저장 완료! 총 수집 상품 수:", len(df))


SyntaxError: invalid syntax (2659183242.py, line 7)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# 설정
chrome_path = "C:/Users/baby3/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"
options = Options()
options.add_argument("--start-maximized")

# 실행
driver = webdriver.Chrome()
driver.get("https://www.dongwonmall.com/category/main.do?cate_id=01110067")

time.sleep(2)  # 초기 로딩 대기

# 전체 카테고리 링크 및 이름 수집
category_elements = driver.find_elements(By.CSS_SELECTOR, "div#allCateList > div.tree-group > ul.list-depth > li.list-item > a")
category_info = []

for idx, elem in enumerate(category_elements):
    try:
        name = elem.find_element(By.CSS_SELECTOR, "span.txt").text.strip()
        link = elem.get_attribute("href")
        category_info.append({"카테고리명": name, "링크": link})
    except:
        continue

# 출력 확인
for c in category_info:
    print(f"{c['카테고리명']} → {c['링크']}")
    driver.get(c['링크'])
    time.sleep(2)


# (선택) 첫 번째 카테고리 클릭 예시
# if category_elements:
#     print("✅ 첫 카테고리 클릭:", category_info[0]['카테고리명'])
#     driver.execute_script("arguments[0].click();", category_elements[0])
#     time.sleep(3)

# 필요한 작업 추가...

# driver.quit()


 → https://www.dongwonmall.com/category/main.do?cate_id=01110065
 → https://www.dongwonmall.com/category/main.do?cate_id=01110066
 → https://www.dongwonmall.com/category/main.do?cate_id=01110067
 → https://www.dongwonmall.com/category/main.do?cate_id=01110068
 → https://www.dongwonmall.com/category/main.do?cate_id=01110070
 → https://www.dongwonmall.com/category/main.do?cate_id=01110073
 → https://www.dongwonmall.com/category/main.do?cate_id=01110075
 → https://www.dongwonmall.com/category/main.do?cate_id=01110076
 → https://www.dongwonmall.com/category/main.do?cate_id=01110077
 → https://www.dongwonmall.com/category/main.do?cate_id=01110078
 → https://www.dongwonmall.com/category/main.do?cate_id=01110079
 → https://www.dongwonmall.com/category/main.do?cate_id=01110080
 → https://www.dongwonmall.com/category/main.do?cate_id=01110082


In [ ]:
# 출력 확인
for c in category_info:
    print(f"{c['카테고리명']} → {c['링크']}")
    driver.get(c['링크'])
    time.sleep(2)


<selenium.webdriver.remote.webelement.WebElement (session="e426788963cad073fa60bce9d573d30d", element="f.A303BDD0F71DF78C4699BEFEE2694DB7.d.76B245B905CD7D05ED710AABB87F56BE.e.231")>

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# 크롬 드라이버 경로
chrome_path = "C:/Users/baby3/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"
options = Options()
options.add_argument("--start-maximized")

# 드라이버 실행
driver = webdriver.Chrome()
driver.get("https://www.dongwonmall.com/category/main.do?cate_id=01110065")
time.sleep(2)

# 전체 카테고리 목록 수집
category_elements = driver.find_elements(By.CSS_SELECTOR, "div#allCateList > div.tree-group > ul.list-depth > li.list-item > a")
category_info = []

for elem in category_elements:
    try:
        name = elem.find_element(By.CSS_SELECTOR, "span.txt").text.strip()
        link = elem.get_attribute("href")
        category_info.append({"카테고리명": name, "링크": link})
    except:
        continue

print(f"📌 총 {len(category_info)}개 카테고리 수집됨")

# 전체 결과 저장용 리스트
all_data = []

# 카테고리별 순회
for idx, cat in enumerate(category_info):
    print(f"\n🔎 [{idx+1}/{len(category_info)}] 카테고리: {cat['카테고리명']}")

    driver.get(cat['링크'])
    time.sleep(2)

    try:
        # 메인 타이틀 (실제 페이지 카테고리명)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h2.title.mainTitle")))
        main_title = driver.find_element(By.CSS_SELECTOR, "h2.title.mainTitle").text.strip()
    except:
        main_title = cat['카테고리명']

    # "리뷰 많은순" 클릭
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.sort")))
        sort_buttons = driver.find_elements(By.CSS_SELECTOR, "span.sort")
        for btn in sort_buttons:
            if btn.text.strip() == "리뷰 많은순":
                btn.click()
                time.sleep(2)
                break
    except:
        print("⚠️ 리뷰순 정렬 버튼 없음")
        continue

    current_page = 1
    max_pages = 5

    while True:
        print(f"   [페이지 {current_page}]")

        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul#bandBestList > li.product_item"))
            )
            time.sleep(1)
        except:
            print("⚠️ 상품 항목 로드 실패")
            break

        items = driver.find_elements(By.CSS_SELECTOR, "ul#bandBestList > li.product_item")
        for item in items:
            try:
                name = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > a > p.name").text.strip()
            except:
                name = "N/A"
            try:
                price = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > div.price_info > span.new_price > strong").text.strip()
            except:
                price = "N/A"
            try:
                grade = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > span.star_rating > span.grade").text.strip()
            except:
                grade = "N/A"
            try:
                review = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > span.star_rating > span.review").text.strip()
            except:
                review = "N/A"

            all_data.append({
                "카테고리": main_title,
                "페이지": current_page,
                "상품명": name,
                "가격": price,
                "평점": grade,
                "리뷰 수": review
            })

        if current_page >= max_pages:
            break

        try:
            page_selector = f'div.paging.paging_t20r > ul > li > a'
            all_pages = driver.find_elements(By.CSS_SELECTOR, page_selector)
            next_page_number = str(current_page + 1)
            next_page_elem = None
            for p in all_pages:
                if p.text.strip() == next_page_number:
                    next_page_elem = p
                    break

            if next_page_elem:
                driver.execute_script("arguments[0].click();", next_page_elem)
                current_page += 1
                time.sleep(2)
            else:
                break
        except Exception as e:
            print("❌ 페이징 오류:", e)
            break

# 저장
driver.quit()
df = pd.DataFrame(all_data)
file_name = f"dongwonmall_all_categories_reviews.xlsx"
df.to_excel(file_name, index=False)
print(f"\n✅ 전체 카테고리 수집 완료! 총 {len(df)}개 상품 저장됨 → {file_name}")


📌 총 13개 카테고리 수집됨

🔎 [1/13] 카테고리: 
   [페이지 1]
   [페이지 2]
   [페이지 3]
   [페이지 4]
   [페이지 5]

🔎 [2/13] 카테고리: 
   [페이지 1]
   [페이지 2]
   [페이지 3]
   [페이지 4]
   [페이지 5]

🔎 [3/13] 카테고리: 
   [페이지 1]
   [페이지 2]
   [페이지 3]
   [페이지 4]
   [페이지 5]

🔎 [4/13] 카테고리: 
   [페이지 1]
   [페이지 2]
   [페이지 3]
   [페이지 4]
   [페이지 5]

🔎 [5/13] 카테고리: 
   [페이지 1]
   [페이지 2]
   [페이지 3]
   [페이지 4]
   [페이지 5]

🔎 [6/13] 카테고리: 
   [페이지 1]
   [페이지 2]
   [페이지 3]
   [페이지 4]
   [페이지 5]

🔎 [7/13] 카테고리: 
   [페이지 1]
   [페이지 2]
   [페이지 3]
   [페이지 4]
   [페이지 5]

🔎 [8/13] 카테고리: 
   [페이지 1]
   [페이지 2]
   [페이지 3]
   [페이지 4]
   [페이지 5]

🔎 [9/13] 카테고리: 
   [페이지 1]
   [페이지 2]
   [페이지 3]
   [페이지 4]
   [페이지 5]

🔎 [10/13] 카테고리: 
   [페이지 1]
   [페이지 2]
   [페이지 3]
   [페이지 4]
   [페이지 5]

🔎 [11/13] 카테고리: 
   [페이지 1]
   [페이지 2]
   [페이지 3]
   [페이지 4]
   [페이지 5]

🔎 [12/13] 카테고리: 
   [페이지 1]
   [페이지 2]
   [페이지 3]
   [페이지 4]
   [페이지 5]

🔎 [13/13] 카테고리: 
   [페이지 1]
   [페이지 2]
   [페이지 3]
   [페이지 4]
   [페이지 5]

✅ 전체 카테고리 수집 완료! 총 6240개 상품 저장됨 → dongwonm

In [6]:
%pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]

Note: you may need to restart the kernel to use updated packages.


In [79]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import time
import pandas as pd
import traceback


# --- 설정 ---
# 크롬 드라이버가 설치된 경로를 지정하세요.
# 예: "C:/Users/user/chromedriver.exe"
# chrome_path = "YOUR_CHROME_DRIVER_PATH" 
# service = Service(executable_path=chrome_path)
# driver = webdriver.Chrome(service=service)
# 크롬 드라이버 경로
chrome_path = "C:/Users/baby3/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"
options = Options()
options.add_argument("--start-maximized")

# 방문할 최상위 카테고리 URL
driver = webdriver.Chrome()
start_url = "https://www.dongwonmall.com/category/main.do?cate_id=01110065" # 가공식품
driver.get(start_url)
time.sleep(2)

# --- 1차 카테고리 목록 수집 ---

# 좌측 '전체 카테고리' 메뉴에서 1차 카테고리 목록을 가져옵니다.
l1_category_elements = driver.find_elements(By.CSS_SELECTOR, "div#allCateList > div.tree-group > ul.list-depth > li.list-item > a")
l1_category_info = []
for elem in l1_category_elements:
    try:
        name = elem.find_element(By.CSS_SELECTOR, "span.txt").get_attribute("textContent").strip()
        link = elem.get_attribute("href")
        if name and link: # 이름과 링크가 모두 있는 경우에만 추가
            l1_category_info.append({"name": name, "link": link})
    except NoSuchElementException:
        continue

print(f"📌 총 {len(l1_category_info)}개의 1차 카테고리 수집 완료")
print("-" * 50)

# --- 전체 결과 저장용 리스트 ---
all_data = []

# --- 메인 크롤링 루프 ---
# 1. 1차 카테고리 순회
for l1_cat in l1_category_info:
    l1_name = l1_cat['name']
    l1_link = l1_cat['link']
    
    print(f"\n[1차] {l1_name} ({l1_link})")
    driver.get(l1_link)
    time.sleep(2)

    # 2. 2차 카테고리 목록 수집
    # '현재 카테고리' 메뉴에서 2차 카테고리를 찾습니다.
    try:
        l2_category_elements = driver.find_elements(By.CSS_SELECTOR, "div#nowCateList > div.tree-group > ul.list-depth > li.list-item >  ul.list-depth > li.list-item > a")
        l2_category_info = []
        for elem in l2_category_elements:
            # javascript:changeCategory('01110065','수산/건어물','N') 형태의 링크를 처리
            onclick_attr = elem.get_attribute("href")
            nameX = elem.find_element(By.CSS_SELECTOR, "span")
            name = nameX.get_attribute("textContent").strip()

            if name and "changeCategory" in onclick_attr:
                l2_category_info.append({"name": name, "onclick": onclick_attr})
    except NoSuchElementException:
        print(f"  └─ ⚠️ '{l1_name}'에서 2차 카테고리를 찾을 수 없습니다.")
        continue

    # 3. 2차 카테고리 순회
    for l2_idx in range(len(l2_category_info)):
        # StaleElementReferenceException 방지를 위해 매번 요소를 다시 찾습니다.
        try:
            l2_elements = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div#nowCateList > div.tree-group > ul.list-depth > li.list-item >  ul.list-depth > li.list-item > a"))
            )
            l2_cat = l2_elements[l2_idx]
            l2_nameX = l2_cat.find_element(By.CSS_SELECTOR, "span")
            l2_name = l2_nameX.get_attribute("textContent").strip()
            print(f"  └─ [2차] {l2_name}")
            
            # 2차 카테고리 클릭
            l2_cat_onclick = l2_cat.get_attribute("href")
            driver.execute_script(l2_cat_onclick)
            time.sleep(2)
        except (StaleElementReferenceException, IndexError):
            print(f"  └─ ⚠️ 2차 카테고리({l2_name})로 이동 중 오류 발생. 건너뜁니다.")
            driver.get(l1_link) # 1차 카테고리 페이지로 복귀 후 다음 2차 시도
            time.sleep(2)
            continue

        # 4. 3차 카테고리 목록 수집
        try:
            l3_category_elements = driver.find_elements(By.CSS_SELECTOR, "div#nowCateList ul.list-depth.morethan > li > ul.list-depth.morethan > li > a")
            l3_category_info = []
            for elem in l3_category_elements:
                onclick_attr = elem.get_attribute("href")
                nameX = elem.find_element(By.CSS_SELECTOR, "span")
                name = nameX.get_attribute("textContent").strip()
                if name and "changeCategory" in onclick_attr:
                    l3_category_info.append({"name": name, "onclick": onclick_attr})
        except NoSuchElementException:
            print(f"    └─ ⚠️ '{l2_name}'에서 3차 카테고리를 찾을 수 없습니다.")
            continue
            
        # 5. 3차 카테고리 순회 및 데이터 수집
        if not l3_category_info:
            print(f"    └─ ⚠️ 3차 카테고리 없음. 건너뜁니다.")
            continue

        for l3_idx in range(len(l3_category_info)):
            try:
                # StaleElementReferenceException 방지를 위해 3차 카테고리 요소 다시 찾기
                l3_elements = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div#nowCateList ul.list-depth.morethan > li > ul.list-depth.morethan > li > a"))
                )
                l3_cat = l3_elements[l3_idx]
                l3_nameX = l3_cat.find_element(By.CSS_SELECTOR, "span")
                l3_name = l3_nameX.get_attribute("textContent").strip()
                print(f"    └─ [3차] {l3_name} (데이터 수집 시작)")
                
                # 3차 카테고리 클릭
                l3_onclick = l3_cat.get_attribute("href")
                driver.execute_script(l3_onclick)
                time.sleep(2)

                # "리뷰 많은순" 클릭
                try:
                    sort_buttons = WebDriverWait(driver, 5).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.sort"))
                    )
                    review_sort_btn = next((btn for btn in sort_buttons if btn.text.strip() == "리뷰 많은순"), None)
                    if review_sort_btn:
                        review_sort_btn.click()
                        time.sleep(2)
                    else:
                        print("      └─ ⚠️ 리뷰순 정렬 버튼 없음")
                except TimeoutException:
                    print("      └─ ⚠️ 정렬 버튼을 찾을 수 없음")
                    continue
                
                # 6. 페이지 순회하며 상품 정보 수집 (최대 5페이지)
                max_pages = 5
                for page_num in range(1, max_pages + 1):
                    print(f"      └─ 페이지 {page_num}/{max_pages}")
                    try:
                        WebDriverWait(driver, 10).until(
                            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul#bandBestList > li.product_item"))
                        )
                    except TimeoutException:
                        print("        └─ ⚠️ 상품 목록을 불러오지 못했습니다.")
                        break

                    items = driver.find_elements(By.CSS_SELECTOR, "ul#bandBestList > li.product_item")
                    if not items:
                        print("        └─ 상품이 없습니다.")
                        break

                    for item in items:
                        try:
                            name = item.find_element(By.CSS_SELECTOR, "p.name").text.strip()
                            price = item.find_element(By.CSS_SELECTOR, "div.price_info span.new_price strong").text.strip()
                            grade = item.find_element(By.CSS_SELECTOR, "span.star_rating span.grade").text.strip()
                            review = item.find_element(By.CSS_SELECTOR, "span.star_rating span.review").text.strip()
                        except NoSuchElementException:
                            # 일부 정보가 없는 상품은 건너뛰지 않고 N/A로 기록
                            name = name if 'name' in locals() else "N/A"
                            price = price if 'price' in locals() else "N/A"
                            grade = grade if 'grade' in locals() else "N/A"
                            review = review if 'review' in locals() else "N/A"

                        all_data.append({
                            "1차 카테고리": l1_name,
                            "2차 카테고리": l2_name,
                            "3차 카테고리": l3_name,
                            "페이지": page_num,
                            "상품명": name,
                            "가격": price,
                            "평점": grade,
                            "리뷰 수": review
                        })
                    
                    # 다음 페이지로 이동
                    if page_num < max_pages:
                        try:
                            # 현재 페이지 번호 다음 번호를 가진 버튼을 찾아 클릭
                            next_page_link = driver.find_element(By.XPATH, f"//div[contains(@class, 'paging')]//a[text()='{page_num + 1}']")
                            driver.execute_script("arguments[0].click();", next_page_link)
                            time.sleep(2)
                        except NoSuchElementException:
                            print("        └─ 다음 페이지 버튼이 없어 수집을 중단합니다.")
                            break
                
                # 현재 2차 카테고리 페이지로 돌아가서 다음 3차 카테고리 준비
                driver.back()
                time.sleep(2)

            except TimeoutException:
                # ★★★★★ 여기가 핵심 ★★★★★
                print(f"    └─ ⚠️ 3차 카테고리 목록 로드 실패. 페이지 상태 복구를 시도합니다.")
                print(f"       (현재 URL: {driver.current_url})")
                
                # 현재 2차 카테고리 링크를 다시 클릭해서 페이지를 새로고침합니다.
                # l2_cat_onclick 변수는 바깥쪽 2차 카테고리 루프에서 가져와야 합니다.
                # (아래 전체 코드에서 변수 전달 부분을 확인하세요)
                driver.execute_script(l2_cat_onclick)
                time.sleep(2)
                
                # 페이지를 복구했으니, 현재 3차 카테고리부터 다시 시도합니다.
                print("       └─ 페이지 복구 완료. 현재 카테고리부터 재시도합니다.")
                continue

            except (StaleElementReferenceException, IndexError) as e:
                print(f"    └─ ⚠️ 3차 카테고리({l3_name}) 처리 중 다른 오류 발생: {e}")
                traceback.print_exc()
                continue

# --- 저장 ---
driver.quit()
if all_data:
    df = pd.DataFrame(all_data)
    file_name = "dongwonmall_3rd_categories_reviews.xlsx"
    df.to_excel(file_name, index=False)
    print(f"\n✅ 전체 카테고리 수집 완료! 총 {len(df)}개 상품 저장됨 → {file_name}")
else:
    print("\n❌ 수집된 데이터가 없습니다.")

📌 총 13개의 1차 카테고리 수집 완료
--------------------------------------------------

[1차] 통조림/즉석/면류 (https://www.dongwonmall.com/category/main.do?cate_id=01110065)
  └─ [2차] 통조림/캔
    └─ [3차] 참치/연어통조림 (데이터 수집 시작)
      └─ 페이지 1/5
      └─ 페이지 2/5
      └─ 페이지 3/5
      └─ 페이지 4/5
        └─ 다음 페이지 버튼이 없어 수집을 중단합니다.
    └─ [3차] 고등어/꽁치통조림 (데이터 수집 시작)
      └─ 페이지 1/5
        └─ 다음 페이지 버튼이 없어 수집을 중단합니다.
    └─ ⚠️ 3차 카테고리 목록 로드 실패. 페이지 상태 복구를 시도합니다.
       (현재 URL: data:,)


JavascriptException: Message: javascript error: changeCategory is not defined
  (Session info: chrome=138.0.7204.170)
Stacktrace:
	GetHandleVerifier [0x0x7ff6e501e415+77285]
	GetHandleVerifier [0x0x7ff6e501e470+77376]
	(No symbol) [0x0x7ff6e4de9a6a]
	(No symbol) [0x0x7ff6e4df1419]
	(No symbol) [0x0x7ff6e4df4811]
	(No symbol) [0x0x7ff6e4e91c0b]
	(No symbol) [0x0x7ff6e4e6860a]
	(No symbol) [0x0x7ff6e4e9087f]
	(No symbol) [0x0x7ff6e4e683e3]
	(No symbol) [0x0x7ff6e4e31521]
	(No symbol) [0x0x7ff6e4e322b3]
	GetHandleVerifier [0x0x7ff6e5301efd+3107021]
	GetHandleVerifier [0x0x7ff6e52fc29d+3083373]
	GetHandleVerifier [0x0x7ff6e531bedd+3213485]
	GetHandleVerifier [0x0x7ff6e503884e+184862]
	GetHandleVerifier [0x0x7ff6e504055f+216879]
	GetHandleVerifier [0x0x7ff6e5027084+113236]
	GetHandleVerifier [0x0x7ff6e5027239+113673]
	GetHandleVerifier [0x0x7ff6e500e298+11368]
	BaseThreadInitThunk [0x0x7fff9b0fe8d7+23]
	RtlUserThreadStart [0x0x7fff9c5fc34c+44]


In [67]:
for l1_cat in l1_category_info:
    l1_name = l1_cat['name']
    l1_link = l1_cat['link']
    print(f"\n[1차] {l1_name} ({l1_link})")


[1차] 생수/음료/커피 (https://www.dongwonmall.com/category/main.do?cate_id=01110066)

[1차] 과자/떡/베이커리 (https://www.dongwonmall.com/category/main.do?cate_id=01110067)

[1차] 냉장/냉동/간편식 (https://www.dongwonmall.com/category/main.do?cate_id=01110068)

[1차] 유제품 (https://www.dongwonmall.com/category/main.do?cate_id=01110070)

[1차] 김치/반찬/양념 (https://www.dongwonmall.com/category/main.do?cate_id=01110073)

[1차] 과일/채소/쌀 (https://www.dongwonmall.com/category/main.do?cate_id=01110075)

[1차] 수산/건어물 (https://www.dongwonmall.com/category/main.do?cate_id=01110076)

[1차] 축산/계란 (https://www.dongwonmall.com/category/main.do?cate_id=01110077)

[1차] 건강식품 (https://www.dongwonmall.com/category/main.do?cate_id=01110078)

[1차] 주방용품 (https://www.dongwonmall.com/category/main.do?cate_id=01110079)

[1차] 생활/유아동 (https://www.dongwonmall.com/category/main.do?cate_id=01110080)

[1차] 반려동식물 (https://www.dongwonmall.com/category/main.do?cate_id=01110082)


In [1]:
import pandas as pd
import time
import traceback
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- 설정 ---
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
start_url = "https://www.dongwonmall.com/category/main.do?cate_id=01110065" # 최상위 카테고리
driver.get(start_url)
time.sleep(2)

# =======================================================================================
# 1. 네비게이션 정보 전체 수집
# =======================================================================================
print("=========== 1. 전체 카테고리 네비게이션 정보 수집 시작 ===========")
all_navigation_tasks = []

# 1-1. 1차 카테고리 정보 수집
l1_elements = driver.find_elements(By.CSS_SELECTOR, "div#allCateList > div.tree-group > ul.list-depth > li.list-item > a")
l1_category_info = []
for elem in l1_elements:
    try:
        name = elem.find_element(By.CSS_SELECTOR, "span.txt").get_attribute("textContent").strip()
        link = elem.get_attribute("href")
        print(f"  └─ [1차] '{name}' '{link}' 정보 수집 완료.")
        if name and link:
            l1_category_info.append({"name": name, "link": link})
    except NoSuchElementException:
        continue

print(f"📌 총 {len(l1_category_info)}개의 1차 카테고리 발견")

# 1-2. 각 카테고리를 순회하며 최종 3차 카테고리 정보 추출
for l1_cat in l1_category_info:
    l1_name = l1_cat['name']
    l1_link = l1_cat['link']
    
    try:
        print(f"\n[1차] '{l1_name}' 탐색 중...")
        driver.get(l1_link)
        time.sleep(1)

        # 1-3. 2차 카테고리 정보 수집
        l2_elements = driver.find_elements(By.CSS_SELECTOR, "div#nowCateList > div.tree-group > ul.list-depth > li.list-item >  ul.list-depth > li.list-item > a")
        if not l2_elements:
            print(f"  └─ 2차 카테고리 없음.")
            continue
        
        l2_category_info = []
        for elem in l2_elements:
            name = elem.get_attribute("textContent").strip()
            onclick = elem.get_attribute("href")
            if name and "changeCategory" in onclick:
                l2_category_info.append({"name": name, "onclick": onclick})

        # 1-4. 3차 카테고리 정보 수집
        for l2_cat in l2_category_info:
            l2_name = l2_cat['name']
            l2_onclick = l2_cat['onclick']
            print(f"  └─ [2차] '{l2_name}' 탐색 중...")
            
            driver.execute_script(l2_onclick)
            time.sleep(1)

            l3_elements = driver.find_elements(By.CSS_SELECTOR, "div#nowCateList ul.list-depth.morethan > li > ul.list-depth.morethan > li > a")
            if not l3_elements:
                print(f"    └─ 3차 카테고리 없음.")
                continue

            for elem in l3_elements:
                l3_name = elem.get_attribute("textContent").strip()
                l3_onclick = elem.get_attribute("href")
                if l3_name and "changeCategory" in l3_onclick:
                    print(f"      └─ [3차] '{l3_name}' 정보 수집 완료.")
                    # 최종 작업 목록에 추가
                    all_navigation_tasks.append({
                        "l1_name": l1_name, "l1_link": l1_link,
                        "l2_name": l2_name, "l2_onclick": l2_onclick,
                        "l3_name": l3_name, "l3_onclick": l3_onclick
                    })
    except Exception as e:
        print(f"🚨 '{l1_name}' 카테고리 정보 수집 중 오류 발생. 다음으로 넘어갑니다.")
        traceback.print_exc()
        continue

print(f"\n=========== ✅ 총 {len(all_navigation_tasks)}개의 최종 카테고리 정보 수집 완료 ===========")


# =======================================================================================
# 2. 수집된 정보 기반으로 데이터 크롤링
# =======================================================================================
print("\n=========== 2. 실제 데이터 크롤링 시작 ===========")
all_data = []

for i, task in enumerate(all_navigation_tasks):
    print(f"\n🔎 [{i+1}/{len(all_navigation_tasks)}] 작업 시작: {task['l1_name']} > {task['l2_name']} > {task['l3_name']}")
    
    try:
        # 각 작업마다 안정성을 위해 1차 카테고리부터 다시 시작
        driver.get(task['l1_link'])
        time.sleep(1)
        driver.execute_script(task['l2_onclick'])
        time.sleep(1)
        driver.execute_script(task['l3_onclick'])
        time.sleep(2)

        # "리뷰 많은순" 클릭
        try:
            sort_buttons = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.sort")))
            review_sort_btn = next((btn for btn in sort_buttons if btn.text.strip() == "리뷰 많은순"), None)
            if review_sort_btn:
                review_sort_btn.click()
                time.sleep(2)
        except TimeoutException:
            print("  └─ ⚠️ 정렬 버튼을 찾을 수 없음. 그대로 진행합니다.")

        # 페이지 순회하며 상품 정보 수집 (최대 10페이지)
        max_pages = 10
        for page_num in range(1, max_pages + 1):
            print(f"  └─ 페이지 {page_num}/{max_pages}")
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul#bandBestList > li.product_item")))
            except TimeoutException:
                print("    └─ ⚠️ 상품 목록을 불러오지 못했습니다.")
                break

            items = driver.find_elements(By.CSS_SELECTOR, "ul#bandBestList > li.product_item")
            if not items:
                print("    └─ 상품이 없습니다.")
                break

            for item in items:
                try:
                    name = item.find_element(By.CSS_SELECTOR, "p.name").text.strip()
                    price = item.find_element(By.CSS_SELECTOR, "div.price_info span.new_price strong").text.strip()
                    grade = item.find_element(By.CSS_SELECTOR, "span.star_rating span.grade").text.strip()
                    review = item.find_element(By.CSS_SELECTOR, "span.star_rating span.review").text.strip()
                    all_data.append({
                        "1차 카테고리": task['l1_name'], "2차 카테고리": task['l2_name'], "3차 카테고리": task['l3_name'],
                        "페이지": page_num, "상품명": name, "가격": price, "평점": grade, "리뷰 수": review
                    })
                except NoSuchElementException:
                    continue # 일부 정보 없는 상품은 건너뛰기
            
            if page_num < max_pages:
                try:
                    next_page_link = driver.find_element(By.XPATH, f"//div[contains(@class, 'paging')]//a[text()='{page_num + 1}']")
                    driver.execute_script("arguments[0].click();", next_page_link)
                    time.sleep(2)
                except NoSuchElementException:
                    print("    └─ 다음 페이지 버튼이 없어 수집을 중단합니다.")
                    break
    except Exception as e:
        print(f"🚨🚨🚨 작업 [{i+1}] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.")
        traceback.print_exc()
        continue


# --- 저장 ---
driver.quit()
if all_data:
    df = pd.DataFrame(all_data)
    file_name = f"dongwonmall_final_reviews.xlsx"
    df.to_excel(file_name, index=False)
    print(f"\n✅ 전체 크롤링 완료! 총 {len(df)}개 상품 저장됨 → {file_name}")
else:
    print("\n❌ 수집된 데이터가 없습니다.")

=========== 1. 전체 카테고리 네비게이션 정보 수집 시작 ===========
  └─ [1차] '통조림/즉석/면류' 'https://www.dongwonmall.com/category/main.do?cate_id=01110065' 정보 수집 완료.
  └─ [1차] '생수/음료/커피' 'https://www.dongwonmall.com/category/main.do?cate_id=01110066' 정보 수집 완료.
  └─ [1차] '과자/떡/베이커리' 'https://www.dongwonmall.com/category/main.do?cate_id=01110067' 정보 수집 완료.
  └─ [1차] '냉장/냉동/간편식' 'https://www.dongwonmall.com/category/main.do?cate_id=01110068' 정보 수집 완료.
  └─ [1차] '유제품' 'https://www.dongwonmall.com/category/main.do?cate_id=01110070' 정보 수집 완료.
  └─ [1차] '김치/반찬/양념' 'https://www.dongwonmall.com/category/main.do?cate_id=01110073' 정보 수집 완료.
  └─ [1차] '과일/채소/쌀' 'https://www.dongwonmall.com/category/main.do?cate_id=01110075' 정보 수집 완료.
  └─ [1차] '수산/건어물' 'https://www.dongwonmall.com/category/main.do?cate_id=01110076' 정보 수집 완료.
  └─ [1차] '축산/계란' 'https://www.dongwonmall.com/category/main.do?cate_id=01110077' 정보 수집 완료.
  └─ [1차] '건강식품' 'https://www.dongwonmall.com/category/main.do?cate_id=01110078' 정보 수집 완료.
  └─ [1차] '주

Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_4032\49566214.py", line 111, in <module>
    driver.execute_script(task['l3_onclick'])
  File "c:\Users\Admin\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 544, in execute_script
    return self.execute(command, {"script": script, "args": converted_args})["value"]
  File "c:\Users\Admin\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 447, in execute
    self.error_handler.check_response(response)
  File "c:\Users\Admin\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 232, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=138.0.7204.185)
Stacktrace:
	GetHandleVerifier [0x0x7ff6e56ee415+77285]


🚨🚨🚨 작업 [97] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [98/271] 작업 시작: 건강식품 > 영양제 > 프로폴리스
🚨🚨🚨 작업 [98] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [99/271] 작업 시작: 건강식품 > 영양제 > 코엔자임/글루타치온
🚨🚨🚨 작업 [99] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [100/271] 작업 시작: 건강식품 > 영양제 > 기타영양제
🚨🚨🚨 작업 [100] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [101/271] 작업 시작: 건강식품 > 미네랄/자연추출물 > 칼슘
🚨🚨🚨 작업 [101] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [102/271] 작업 시작: 건강식품 > 미네랄/자연추출물 > 마그네슘
🚨🚨🚨 작업 [102] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [103/271] 작업 시작: 건강식품 > 미네랄/자연추출물 > 아연
🚨🚨🚨 작업 [103] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [104/271] 작업 시작: 건강식품 > 미네랄/자연추출물 > 엽산/철분
🚨🚨🚨 작업 [104] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [105/271] 작업 시작: 건강식품 > 미네랄/자연추출물 > MSM/보스웰리아
🚨🚨🚨 작업 [105] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [106/271] 작업 시작: 건강식품 > 미네랄/자연추출물 > 차전자피/식이섬유
🚨🚨🚨 작업 [106] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [107/271] 작업 시작: 건강식품 > 미네랄/자연추출물 > 자연추출물/기타미네랄
🚨🚨🚨 작업 [107] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [108/271] 작업 시작: 건강식품 > 건강즙/건강주스 > 양배추즙
🚨🚨🚨 작업 [108] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [109/271] 작

Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_4032\49566214.py", line 107, in <module>
    driver.get(task['l1_link'])
  File "c:\Users\Admin\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 472, in get
    self.execute(Command.GET, {"url": url})
  File "c:\Users\Admin\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 447, in execute
    self.error_handler.check_response(response)
  File "c:\Users\Admin\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 232, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x0x7ff6e56ee415+77285]
	GetHandleVerifier [0x0x7ff6e56ee470+77376]
	(No symbol) [0x0x7ff6e54b989c]
	(No symbol) [0x0x7ff6e550064f]
	(No symbol) [0x0x7ff6e55384d2]
	(No symbol) [0x0x7ff6e5532fd3]
	(No symbol) [0x0x7ff6e5532099]
	(No s


🔎 [237/271] 작업 시작: 반려동식물 > 고양이 용품 > 낚시/막대/장난감
🚨🚨🚨 작업 [237] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [238/271] 작업 시작: 반려동식물 > 고양이 용품 > 식기/급수기
🚨🚨🚨 작업 [238] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [239/271] 작업 시작: 반려동식물 > 고양이 용품 > 미용/목욕/위생
🚨🚨🚨 작업 [239] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [240/271] 작업 시작: 반려동식물 > 고양이 용품 > 이동장/캐리어
🚨🚨🚨 작업 [240] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [241/271] 작업 시작: 반려동식물 > 고양이 용품 > 패션용품
🚨🚨🚨 작업 [241] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [242/271] 작업 시작: 반려동식물 > 고양이 용품 > 기타
🚨🚨🚨 작업 [242] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [243/271] 작업 시작: 반려동식물 > 강아지 식품 > 건식사료
🚨🚨🚨 작업 [243] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [244/271] 작업 시작: 반려동식물 > 강아지 식품 > 습식사료
🚨🚨🚨 작업 [244] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [245/271] 작업 시작: 반려동식물 > 강아지 식품 > 덴탈/껌
🚨🚨🚨 작업 [245] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [246/271] 작업 시작: 반려동식물 > 강아지 식품 > 사사미/육포
🚨🚨🚨 작업 [246] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [247/271] 작업 시작: 반려동식물 > 강아지 식품 > 캔/파우치
🚨🚨🚨 작업 [247] 처리 중 심각한 오류 발생. 이 작업을 건너뜁니다.

🔎 [248/271] 작업 시작: 반려동식물 > 강아지 식품 > 동결건조/수제간식
🚨🚨🚨 작업 [248] 처

Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_4032\49566214.py", line 107, in <module>
    driver.get(task['l1_link'])
  File "c:\Users\Admin\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 472, in get
    self.execute(Command.GET, {"url": url})
  File "c:\Users\Admin\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 447, in execute
    self.error_handler.check_response(response)
  File "c:\Users\Admin\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 232, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x0x7ff6e56ee415+77285]
	GetHandleVerifier [0x0x7ff6e56ee470+77376]
	(No symbol) [0x0x7ff6e54b989c]
	(No symbol) [0x0x7ff6e550064f]
	(No symbol) [0x0x7ff6e55384d2]
	(No symbol) [0x0x7ff6e5532fd3]
	(No symbol) [0x0x7ff6e5532099]
	(No s


✅ 전체 크롤링 완료! 총 12457개 상품 저장됨 → dongwonmall_final_reviews.xlsx


In [18]:
import pandas as pd
import re

# 엑셀 파일 불러오기
file_path = "모든 카테고리.xlsx"
df = pd.read_excel(file_path, header=None)

# 상품명은 E열 (index 4), 2번째 행부터
product_names = df.iloc[1:, 4]

# 1. 중량 수치, 단위 / 수량, 단위 추출
def parse_product_info_v3(text):
    text = str(text)

    # 예: 150g x10캔, 85g 20캔, 200ml*2병 등
    pattern = re.compile(r"(\d+)\s*(g|kg|ml|L|ℓ)\s*[x×*]?\s*(\d+)?\s*([개캔병팩봉매롤]+)?", re.IGNORECASE)

    weight_vals = []
    weight_units = []
    count_vals = []
    count_units = []

    for match in pattern.findall(text):
        weight_val, weight_unit, count, count_unit = match
        weight_vals.append(weight_val)
        weight_units.append(weight_unit)
        count_vals.append(count if count else "")
        count_units.append(count_unit if count_unit else "")

    return (
        ", ".join(weight_vals),
        ", ".join(weight_units),
        ", ".join(count_vals),
        ", ".join(count_units)
    )

# 결과 리스트
all_weight_vals, all_weight_units = [], []
all_count_vals, all_count_units = [], []

for name in product_names:
    wv, wu, cv, cu = parse_product_info_v3(name)
    all_weight_vals.append(wv)
    all_weight_units.append(wu)
    all_count_vals.append(cv)
    all_count_units.append(cu)

# F ~ I열에 삽입
df.iloc[1:, 5] = all_weight_vals     # 중량 수치
df.iloc[1:, 6] = all_weight_units    # 중량 단위
df.iloc[1:, 7] = all_count_vals      # 수량
df.iloc[1:, 8] = all_count_units     # 수량 단위

# 2. 가격 숫자화 ("49,200원" → 49200)
def clean_price(text):
    text = str(text)
    digits = re.sub(r"[^\d]", "", text)
    return int(digits) if digits else ""

df.iloc[1:, 9] = df.iloc[1:, 9].apply(clean_price)  # J열 (가격)

# 3. 리뷰 수 정제 ("(1,467건)" → 1467) → L열 (index 11)
def extract_review_count(text):
    text = str(text)
    digits = re.sub(r"[^\d]", "", text)
    return int(digits) if digits else ""

df.iloc[1:, 11] = df.iloc[1:, 11].apply(extract_review_count)  # L열 (리뷰 수)

# 평점은 K열 (index 10) → 그대로 유지 (float이므로 처리 안 함)

# 저장
output_path = "모든_카테고리_최종_열구조.xlsx"
df.to_excel(output_path, index=False, header=False)

print("✅ 엑셀 저장 완료:", output_path)


✅ 엑셀 저장 완료: 모든_카테고리_최종_열구조.xlsx
